# Interact with `SampleGen` simulation

requires litex_server running and connected to the VC707 USB-uart port:

```bash
# both run at the same time in separate windows ...
$ python3 sim_sample_gen.py
$ litex_server --uart --uart-port socket://localhost:1111
```

In [1]:
import sys
from numpy import *
from time import sleep
from litex import RemoteClient

def getId(r):
    s = ""
    for i in range(64):
        temp = r.read(r.bases.identifier_mem + i * 4)
        if temp == 0:
            break
        s += chr(temp & 0xFF)
    return s


def hd(dat, pad_width=1, word_width=None):
    ''' print a hex-dump, word_width in bytes '''
    if word_width is None:
        word_width = pad_width
    for i, d in enumerate(dat):
        if i % 8 == 0 and len(dat) > 8:
            print('\n{:04x}: '.format(i * word_width), end='')
        print('{:0{ww}x} '.format(d, ww=pad_width * 2), end='')
    print()
    
def big_write(r, addr, datas, chunk_size=255):
    '''
    write large amount of data to litex server in chunks
    
    r: a litex RemoteClient object
    addr: start address in [bytes]
    datas: list or numpy array of dtype uint32 
    '''
    datas = array(datas, dtype=uint32)
    s_index = 0
    while True:
        dat = datas[s_index: s_index + chunk_size]
        if len(dat) <= 0:
            break
#         print("****", hex(addr))
#         hd(dat, 4)
        r.write(addr, dat.tolist())
        addr += 4 * len(dat)
        s_index += chunk_size

def setSamples(r, samples):
    '''
    Write sample memory of arbitrary waveform generator
  
    r: a litex RemoteClient object
    samples: numpy array of dtype int16
             len(samples) will be clipped to the next multiple of 16
    '''
    N_MEM = 8  # Number of block rams
    N_SAMPLES = 16  # parallel samples / dsp clock cycle

    # signed 16 bit samples in natural order
    samples = array(samples, dtype=int16)

    # pack 2 x signed 16 bit samples into one unsigned 32 bit memory word
    s_u8 = samples.tobytes()
    s_u32 = frombuffer(s_u8, dtype=uint32)

    for n in range(N_MEM):
        mem = getattr(r.mems, f'm0_n{n}')
        s = s_u32[n::N_MEM]
#         hd(s, 4)
        big_write(r, mem.base, s)

    r.regs.sample_gen_max_ind.write(len(samples) // N_SAMPLES - 1)

In [2]:
# Connect to litex server
r = RemoteClient(csr_csv='out/csr.csv', debug=False, port=1234)
r.open()
getId(r)

'LiteX Simulation 2021-03-11 09:36:21'

In [3]:
# samples = arange(0x80) - 0x40
samples = arange(0x2000) - 0x1000

setSamples(r, samples)

In [4]:
# start vcd dump for a few ms, then stop sim.
r.regs.sim_trace_enable.write(1)
sleep(0.01)
r.regs.sim_finish_finish.write(1)

to inspect result, run

```bash
gtkwave out/gateware/sim.vcd
```

look for `TOP/sim/source_converter0` which is the sample stream for the DAC.